In [2]:
import os
import pandas as pd
from openbabel import pybel
from freesasa import Structure, calc
from rdkit import Chem
from pathlib import Path

In [ ]:
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#configurations!!update before running
PROTEIN_STRUC = "./1443.pdb"
LIGANDS_DIR = "./ligand"
OUTPUT_CSV = "./rsa_results.csv"
COMPLEX_DIR = "./complex"

In [4]:
ligand_files = [f for f in os.listdir(LIGANDS_DIR) if f.endswith(('.pdbqt'))]
len(ligand_files)

2143

In [5]:
def build_complex(protein_file, ligand_file, output_file):
    try:
        protein = next(pybel.readfile("pdb", protein_file))
        ligand = next(pybel.readfile("pdbqt", ligand_file))

        complex_mol = pybel.ob.OBMol()
        complex_mol += protein.OBMol
        complex_mol += ligand.OBMol
        complex_pybel = pybel.Molecule(complex_mol)

        complex_pybel.write("pdb", output_file, overwrite=True)
        return True
    except Exception as e:
        print(f"Error building complex for {ligand_file}: {e}")
        return False

In [17]:
complex = "./complex/HIT100003398_complex.pdb"
structure = Structure(complex, options={"hetatm":True})
result = calc(structure)
result.residueAreas()['A']["1"].sideChain

6.59456492521028

In [6]:
results = []
failed = 0
success = 0
for i, ligand_file in enumerate(ligand_files):
    ligand_path = os.path.join(LIGANDS_DIR, ligand_file)
    ligand_name = os.path.splitext(ligand_file)[0]
    complex_pdb = os.path.join(COMPLEX_DIR, f"{ligand_name}_complex.pdb")

    print(f"processing ligand {i+1}/{len(ligand_files)}: {ligand_name}")
    build_complex(PROTEIN_STRUC, ligand_path, complex_pdb)
    
    try:
        structure = Structure(complex_pdb, options={"hetatm":True})
    except Exception as e:
        print(f"error reading complex structure for {ligand_name}: {e}")
        continue
    
    result = calc(structure)
    if result.residueAreas()['A']['1'].relativeTotal is None:
        print(f"failed to calculate RSA for {ligand_name}")
        failed = failed + 1
    else:
        results.append({
        'Ligand': ligand_name,
        'Total_RSA' : result.residueAreas()['A']['1'].relativeTotal,
        'Side_chain_RSA' : result.residueAreas()['A']['1'].relativeSideChain
        })
        success = success + 1
print(f"RSA calculation completed: {len(ligand_files)} total, {success} successful, {failed} failed.")


processing ligand 1/2143: HIT100003398
processing ligand 2/2143: HIT100004257
processing ligand 3/2143: HIT100005077
processing ligand 4/2143: HIT100011477
processing ligand 5/2143: HIT100016598
processing ligand 6/2143: HIT100016832
processing ligand 7/2143: HIT100018045
processing ligand 8/2143: HIT100031508
processing ligand 9/2143: HIT100032652
processing ligand 10/2143: HIT100034699
processing ligand 11/2143: HIT100034939
processing ligand 12/2143: HIT100035980
processing ligand 13/2143: HIT100045194
processing ligand 14/2143: HIT100046453
processing ligand 15/2143: HIT100049001
processing ligand 16/2143: HIT100054036
processing ligand 17/2143: HIT100057403
processing ligand 18/2143: HIT100064812
processing ligand 19/2143: HIT100067522
processing ligand 20/2143: HIT100069145
processing ligand 21/2143: HIT100074326
processing ligand 22/2143: HIT100078960
processing ligand 23/2143: HIT100086251
processing ligand 24/2143: HIT100094941
processing ligand 25/2143: HIT100095173
processin

In [7]:
apo_structure = Structure("./1443.pdb")
apo_result = calc(apo_structure)

results.append({
    'Ligand': 'apo',
    'Total_RSA': apo_result.residueAreas()['A']["104"].relativeTotal,
    'Side_chain_RSA': apo_result.residueAreas()['A']["104"].relativeSideChain
})
results
#print(apo_result.residueAreas()['A']["104"].relativeTotal,apo_result.residueAreas()['A']["104"].relativeSideChain)

[{'Ligand': 'HIT100003398',
  'Total_RSA': 0.31139242766741065,
  'Side_chain_RSA': 0.040845865129825215},
 {'Ligand': 'HIT100004257',
  'Total_RSA': 0.32877747087729847,
  'Side_chain_RSA': 0.04936356807086768},
 {'Ligand': 'HIT100005077',
  'Total_RSA': 0.3289557076643805,
  'Side_chain_RSA': 0.053436759795762866},
 {'Ligand': 'HIT100011477',
  'Total_RSA': 0.31634631118585954,
  'Side_chain_RSA': 0.04274776378583585},
 {'Ligand': 'HIT100016598',
  'Total_RSA': 0.33151791467296476,
  'Side_chain_RSA': 0.05167748065195937},
 {'Ligand': 'HIT100016832',
  'Total_RSA': 0.3312333770183961,
  'Side_chain_RSA': 0.052180574604598164},
 {'Ligand': 'HIT100018045',
  'Total_RSA': 0.32087800840174036,
  'Side_chain_RSA': 0.047222983013415085},
 {'Ligand': 'HIT100031508',
  'Total_RSA': 0.32835872918511744,
  'Side_chain_RSA': 0.049236263796969694},
 {'Ligand': 'HIT100032652',
  'Total_RSA': 0.3017796275611282,
  'Side_chain_RSA': 0.03829140730988567},
 {'Ligand': 'HIT100034699',
  'Total_RSA': 0

In [10]:
# write results to CSV
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by = 'Side_chain_RSA', ascending=True)
results_df.columns = ['vina_title', 'Total_RSA', 'Side_chain_RSA']
results_df['vina_title'] = results_df['vina_title'] + ".pdbqt"
results_df.to_csv(OUTPUT_CSV, index=False)
#print(f"Wrote RSA results to {OUTPUT_CSV}")


In [16]:
df_map = pd.read_csv('./1443_pharm_select.csv')
rsa_select = df_map.merge(results_df.copy(), 
    on = 'vina_title',
    how='left',
    suffixes=('', '_dup'))
rsa_select

,standardized_smiles,vina_title,glide_title,rmsd,score_vina,score_glide,PharmFit_score,Total_RSA,Side_chain_RSA
0,Cc1nn(CC(=O)N2CCC3(CC2)OCCO3)c(=O)c2c(-n3cccc3...,HIT105860007.pdbqt,ligprep_chemdiv-stock1_02_400000_split-10_8.sd...,1.260696,-7.9,-6.01693,2.836370,0.317932,0.044168
1,CCOc1cc(C2CC(=O)Nc3nc(SCc4ccccc4F)[nH]c(=O)c32...,HIT100531985.pdbqt,ligprep_chemdiv-stock1_02_400000_split-10_0.sd...,1.576517,-7.8,-6.68465,2.805660,0.319012,0.038291
2,Cc1cccc(-c2c3c(=O)n(C)c(=O)n(C)c3c3n2-c2ccccc2...,HIT102285018.pdbqt,ligprep_chemdiv-stock1_02_400000_split-10_0.sd...,0.624294,-8.2,-6.26076,2.721380,0.344806,0.065206
3,CCOc1ccc(C2CC(=O)Nc3nc(SCc4ccccc4)[nH]c(=O)c32...,HIT104280749.pdbqt,ligprep_chemdiv-stock1_02_400000_split-10_0.sd...,2.024323,-8.0,-6.49891,2.678040,0.319485,0.038291
4,O=C1CC(c2ccc(F)c(Br)c2)c2c(nc(SCc3ccccc3)[nH]c...,HIT107574755.pdbqt,ligprep_chemdiv-stock1_02_400000_split-10_1.sd...,1.512527,-8.4,-6.20592,2.629000,0.319191,0.038291
...,...,...,...,...,...,...,...,...,...
2139,Cc1ccc(-n2c(-c3cccc([N+](=O)[O-])c3)nc(=S)c3c2...,HIT100747872.pdbqt,ligprep_chemdiv-stock1_01_400000_split-10_2.sd...,0.541712,-7.7,-6.49692,0.502771,0.327821,0.048786
2140,CNC(=O)C1CN(C(=O)c2ccncn2)CC12CCN(c1ccc(C#N)c(...,HIT103047606.pdbqt,ligprep_chemdiv-stock1_03_400000_split-10_8.sd...,2.074108,-8.2,-5.91642,0.502647,0.305068,0.039088
2141,O=C(Cn1c(=O)c(=O)n(Cc2ccncc2)c2ncccc21)N1CCN(c...,HIT104698140.pdbqt,ligprep_chemdiv-stock1_02_400000_split-10_9.sd...,2.094927,-8.1,-5.89543,0.502169,0.317966,0.042929
2142,O=C1c2[nH]nc(-c3ccccc3O)c2C(c2cccc(OCc3ccccc3)...,HIT101305057.pdbqt,ligprep_chemdiv-stock1_01_400000_split-10_9.sd...,1.795784,-8.2,-6.38039,0.501502,0.325137,0.045199


In [17]:
#rsa_select['Side_chain_RSA'] = rsa_select['Side_chain_RSA'].to_numeric()

rsa_select = rsa_select[rsa_select['Side_chain_RSA']<=0.07]
rsa_select = rsa_select[rsa_select['Total_RSA']<=0.35]
rsa_select

,standardized_smiles,vina_title,glide_title,rmsd,score_vina,score_glide,PharmFit_score,Total_RSA,Side_chain_RSA
0,Cc1nn(CC(=O)N2CCC3(CC2)OCCO3)c(=O)c2c(-n3cccc3...,HIT105860007.pdbqt,ligprep_chemdiv-stock1_02_400000_split-10_8.sd...,1.260696,-7.9,-6.01693,2.836370,0.317932,0.044168
1,CCOc1cc(C2CC(=O)Nc3nc(SCc4ccccc4F)[nH]c(=O)c32...,HIT100531985.pdbqt,ligprep_chemdiv-stock1_02_400000_split-10_0.sd...,1.576517,-7.8,-6.68465,2.805660,0.319012,0.038291
2,Cc1cccc(-c2c3c(=O)n(C)c(=O)n(C)c3c3n2-c2ccccc2...,HIT102285018.pdbqt,ligprep_chemdiv-stock1_02_400000_split-10_0.sd...,0.624294,-8.2,-6.26076,2.721380,0.344806,0.065206
3,CCOc1ccc(C2CC(=O)Nc3nc(SCc4ccccc4)[nH]c(=O)c32...,HIT104280749.pdbqt,ligprep_chemdiv-stock1_02_400000_split-10_0.sd...,2.024323,-8.0,-6.49891,2.678040,0.319485,0.038291
4,O=C1CC(c2ccc(F)c(Br)c2)c2c(nc(SCc3ccccc3)[nH]c...,HIT107574755.pdbqt,ligprep_chemdiv-stock1_02_400000_split-10_1.sd...,1.512527,-8.4,-6.20592,2.629000,0.319191,0.038291
...,...,...,...,...,...,...,...,...,...
2139,Cc1ccc(-n2c(-c3cccc([N+](=O)[O-])c3)nc(=S)c3c2...,HIT100747872.pdbqt,ligprep_chemdiv-stock1_01_400000_split-10_2.sd...,0.541712,-7.7,-6.49692,0.502771,0.327821,0.048786
2140,CNC(=O)C1CN(C(=O)c2ccncn2)CC12CCN(c1ccc(C#N)c(...,HIT103047606.pdbqt,ligprep_chemdiv-stock1_03_400000_split-10_8.sd...,2.074108,-8.2,-5.91642,0.502647,0.305068,0.039088
2141,O=C(Cn1c(=O)c(=O)n(Cc2ccncc2)c2ncccc21)N1CCN(c...,HIT104698140.pdbqt,ligprep_chemdiv-stock1_02_400000_split-10_9.sd...,2.094927,-8.1,-5.89543,0.502169,0.317966,0.042929
2142,O=C1c2[nH]nc(-c3ccccc3O)c2C(c2cccc(OCc3ccccc3)...,HIT101305057.pdbqt,ligprep_chemdiv-stock1_01_400000_split-10_9.sd...,1.795784,-8.2,-6.38039,0.501502,0.325137,0.045199


In [18]:
rsa_select.to_csv('./1443_rsa_select.csv', index=False)